# 🏆 TrustChain Pro — Complete Feature Reference

**All TrustChain v2.1 capabilities in one place:**

| Section | Features |
|---------|----------|
| Core | Signing, Verification, Nonces |
| Chain | Parent signatures, Merkle trees |
| Enterprise | Tenants, Metrics, Caching |
| Integrations | LangChain, MCP, OpenAI schemas |
| Security | Key rotation, Replay protection |
| Export | HTML/JSON audit, ChainExplorer |

In [ ]:
# Full imports
import asyncio
import json
import os
import tempfile
import time
from pprint import pprint

from trustchain import (
    TenantManager,
    TrustChain,
    TrustChainConfig,
    get_logger,
    get_metrics,
    setup_logging,
)
from trustchain.v2.merkle import MerkleTree, verify_proof

print("✅ All TrustChain modules loaded")

---
## 🔐 1. Core Signing Engine

In [ ]:
# Full config options
config = TrustChainConfig(
    algorithm="ed25519",        # Signature algorithm
    enable_nonce=True,          # Replay protection
    enable_cache=True,          # Verification cache
    enable_metrics=True,        # Prometheus metrics
    cache_ttl=300,              # Cache TTL seconds
    nonce_ttl=3600,             # Nonce TTL seconds
    nonce_backend="memory",     # memory or redis
)

tc = TrustChain(config)

print("📋 Config:")
print(f"   Algorithm: {config.algorithm}")
print(f"   Nonce: {config.enable_nonce}")
print(f"   Cache: {config.enable_cache}")
print(f"   Metrics: {config.enable_metrics}")
print(f"\n🔑 Key ID: {tc.get_key_id()}")
print(f"📤 Public Key: {tc.export_public_key()[:32]}...")

---
## 🛠️ 2. Tool Decorator API

In [ ]:
# Sync tool
@tc.tool("calculator")
def calculate(expression: str, precision: int = 2):
    """Evaluate mathematical expression."""
    try:
        result = round(eval(expression), precision)
        return {"expression": expression, "result": result}
    except Exception as e:
        return {"error": str(e)}

# Async tool
@tc.tool("api_call")
async def fetch_data(endpoint: str, timeout: int = 30):
    """Fetch data from API endpoint."""
    await asyncio.sleep(0.1)  # Simulate API call
    return {
        "endpoint": endpoint,
        "status": 200,
        "data": {"key": "value"},
        "latency_ms": 95
    }

# Test both
sync_result = calculate("2 ** 10")
async_result = await fetch_data("/api/v1/users")

print("🔢 Sync:", sync_result.data)
print("🌐 Async:", async_result.data)
print(f"\n✅ Both verified: sync={sync_result.is_verified}, async={async_result.is_verified}")

---
## ⛓️ 3. Chain of Trust & Merkle

In [ ]:
# Parent-child chain
step1 = tc._signer.sign("step1", {"action": "init"})
step2 = tc._signer.sign("step2", {"action": "process"}, parent_signature=step1.signature)
step3 = tc._signer.sign("step3", {"action": "commit"}, parent_signature=step2.signature)

print("⛓️ Linked Chain:")
print("   Step1 → Step2 → Step3")
print(f"   Step2.parent[:16] = {step2.parent_signature[:16]}")
print(f"   Step1.sig[:16]    = {step1.signature[:16]}")
print(f"   Match: {step2.parent_signature == step1.signature}")

# Merkle tree
chunks = ["document_page_1", "document_page_2", "document_page_3", "document_page_4"]
tree = MerkleTree.from_chunks(chunks)

print("\n🌳 Merkle Tree:")
print(f"   Leaves: {len(tree.leaves)}")
print(f"   Root: {tree.root[:32]}...")

# Verify single chunk without full document
proof = tree.get_proof(1)  # Get proof for page 2
is_valid = verify_proof("document_page_2", proof, tree.root)
print(f"   Chunk 1 valid: {is_valid}")

---
## 🏢 4. Multi-Tenancy

In [ ]:
with tempfile.TemporaryDirectory() as key_dir:
    manager = TenantManager(
        key_storage_dir=key_dir,  # Persist keys per tenant
    )

    # Create tenants
    tenants = {
        name: manager.get_or_create(name)
        for name in ["acme_corp", "globex_inc", "initech_llc"]
    }

    print("🏢 Enterprise Tenants:")
    for name, tc in tenants.items():
        print(f"   {name}: {tc.get_key_id()[:20]}...")

    print(f"\n📊 Total: {manager.count} tenants")

    # Cross-tenant isolation test
    acme_data = tenants["acme_corp"]._signer.sign("order", {"id": 123})
    globex_can_verify = tenants["globex_inc"].verify(acme_data)
    print(f"\n🔒 Isolation: Globex verify Acme = {globex_can_verify}")

---
## 📊 5. Metrics & Monitoring

In [ ]:
# Enable logging
setup_logging(level="INFO")
logger = get_logger("trustchain.demo")

# Perform operations
tc_metrics = TrustChain(TrustChainConfig(enable_metrics=True, enable_nonce=False))

@tc_metrics.tool("monitored_tool")
def monitored_operation(x: int):
    return {"result": x * 2}

for i in range(10):
    monitored_operation(i)

# Get metrics
metrics = get_metrics()
print("📊 Metrics:")
pprint(metrics)

---
## 🔄 6. Key Management

In [ ]:
with tempfile.TemporaryDirectory() as tmpdir:
    key_path = os.path.join(tmpdir, "production_keys.json")

    # Create with persistent keys
    tc_prod = TrustChain(TrustChainConfig(
        key_file=key_path,
        enable_nonce=False
    ))

    old_key = tc_prod.get_key_id()
    tc_prod.save_keys()
    print(f"💾 Keys saved to: {key_path}")
    print(f"🔑 Current key: {old_key[:20]}...")

    # Rotate keys - generates new keypair
    new_key = tc_prod.rotate_keys()
    print("\n🔄 Key rotated!")
    print(f"🔑 New key: {new_key[:20]}...")

    # Show key file structure
    with open(key_path) as f:
        key_data = json.load(f)
    print(f"\n📄 Key file structure: {list(key_data.keys())}")


---
## 🔗 7. Integrations

In [ ]:
tc_int = TrustChain(TrustChainConfig(enable_nonce=False))

@tc_int.tool("search")
def search_documents(query: str, limit: int = 10):
    """Search documents in knowledge base."""
    return {"query": query, "results": [], "count": 0}

# OpenAI-compatible schema
print("📋 OpenAI Function Schema:")
schema = tc_int.get_tool_schema("search")
print(json.dumps(schema, indent=2))

# LangChain integration
try:
    from trustchain.integrations.langchain import to_langchain_tool
    lc_tool = to_langchain_tool(tc_int, "search")
    print(f"\n🦜 LangChain tool: {lc_tool.name}")
except ImportError:
    print("\n⚠️ LangChain not installed")

# MCP integration
try:
    from trustchain.integrations.mcp import create_mcp_server
    server = create_mcp_server(tc_int, "demo-server")
    print(f"🔌 MCP Server: {server.name}")
except ImportError:
    print("⚠️ MCP SDK not installed")

---
## 📤 8. Export & Audit Trail

In [ ]:
from trustchain.ui.explorer import ChainExplorer

tc_audit = TrustChain(TrustChainConfig(enable_nonce=False))

@tc_audit.tool("audit_action")
def audited_action(action: str, user: str):
    return {"action": action, "user": user, "ts": time.time()}

# Collect audit trail
trail = [
    audited_action("login", "alice"),
    audited_action("view_report", "alice"),
    audited_action("export_data", "alice"),
    audited_action("logout", "alice"),
]

# Create explorer
explorer = ChainExplorer(trail, tc_audit)

# Export HTML
html_path = explorer.export_html("full_audit.html")
print(f"📊 HTML report: {html_path}")

# Export JSON
json_data = explorer.to_json()
parsed = json.loads(json_data)
print(f"📄 JSON: {len(parsed['responses'])} entries")

# Summary stats
stats = explorer.get_stats()
print("\n📈 Stats:")
for k, v in stats.items():
    print(f"   {k}: {v}")

---
## 🛡️ 9. Security Features

In [ ]:
from trustchain.utils.exceptions import NonceReplayError

tc_sec = TrustChain(TrustChainConfig(
    enable_nonce=True,
    nonce_ttl=10  # Short TTL for demo
))

@tc_sec.tool("secure_op")
def secure_operation(data: str):
    return {"data": data, "processed": True}

# Normal operation
result = secure_operation("test")
print(f"✅ Operation executed, nonce: {result.nonce[:16]}...")

# Replay attack simulation
print("\n🦹 Simulating replay attack...")
try:
    # Try to verify same response again (replay)
    tc_sec.verify(result)  # First verify consumes nonce
    tc_sec.verify(result)  # Second should fail
except NonceReplayError as e:
    print(f"🛡️ Replay blocked: {e}")

# Tampering detection
print("\n🦹 Simulating tampering...")
import copy

tc_tamper = TrustChain(TrustChainConfig(enable_nonce=False))

@tc_tamper.tool("balance")
def get_balance():
    return {"balance": 100}

original = get_balance()
tampered = copy.deepcopy(original)
tampered.data["balance"] = 999999

print(f"   Original: {original.data['balance']} → verified={tc_tamper.verify(original)}")
print(f"   Tampered: {tampered.data['balance']} → verified={tc_tamper.verify(tampered)}")

---
## ⚡ 10. Performance Benchmarks

In [ ]:
tc_perf = TrustChain(TrustChainConfig(
    enable_nonce=False,
    enable_cache=False,
    enable_metrics=False
))

@tc_perf.tool("bench")
def benchmark_tool(x: int):
    return {"x": x}

# Signing benchmark
iterations = 1000
start = time.perf_counter()
for i in range(iterations):
    benchmark_tool(i)
sign_time = (time.perf_counter() - start) * 1000

# Verification benchmark
result = benchmark_tool(0)
start = time.perf_counter()
for _ in range(iterations):
    tc_perf.verify(result)
verify_time = (time.perf_counter() - start) * 1000

print("⚡ Performance Benchmarks:")
print(f"   Signing: {sign_time/iterations:.3f}ms/op ({iterations*1000/sign_time:.0f} ops/sec)")
print(f"   Verify:  {verify_time/iterations:.3f}ms/op ({iterations*1000/verify_time:.0f} ops/sec)")
print("\n🏆 Ed25519 delivers consistent sub-millisecond performance!")

---
## 📚 Complete API Reference

| Class | Key Methods |
|-------|-------------|
| `TrustChain` | `.tool()`, `.verify()`, `.get_key_id()`, `._signer.rotate()  # Regenerate keys` |
| `TrustChainConfig` | All configuration options |
| `TrustChainVerifier` | Standalone verification |
| `SignedResponse` | `.data`, `.signature`, `.nonce`, `.is_verified` |
| `TenantManager` | `.get_or_create()`, `.list_tenants()` |
| `MerkleTree` | `.from_chunks()`, `.get_proof()`, `.verify_chunk()` |
| `ChainExplorer` | `.export_html()`, `.to_json()`, `.get_stats()` |

**Documentation:** [GitHub Wiki](https://github.com/petro1eum/trust_chain/wiki)